# Applications

*Taux de rente*\
On trace le graphe de la fonction $f(I) = M - v \frac{1+I}{I}\left[(1+I)^n -1\right]$


In [0]:
import numpy as np
import plotly.graph_objs as go

# Define the function
f = lambda x: 6000 - 1000 * (1 + x) * ((1 + x)**5 - 1) / x

# Define the x range
x = np.arange(0.01, 0.3, 0.001)

# Calculate y values
y = f(x)

# Create the plot
fig = go.Figure(data=go.Scatter(x=x, y=y, mode='lines'))

# Add grid and customize layout
fig.update_layout(
    title='Plot of f(x)',
    xaxis_title='x',
    yaxis_title='f(x)',
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True)
)

# Show the plot
fig.show()


On voit que le zéro est entre $0.05$ et $0.1$.

Si on applique la méthode de dichotomie sur l’intervalle [0.05,0.1] avec
une tolérance de $10^{-5}$


In [0]:
from tan.eqnonlin.bisection import bisection

zero,res,niter,inc=bisection(f,0.05,0.1,1e-5,1000)
print(f"Zero: {zero}, Residual: {res}, Iterations: {niter}")


on trouve, après 12 itérations, la valeur $\alpha = 0.061407470703125$.

Au contraire, en utilisant la méthode de Newton avec la même tolérance à partir de la valeur $x^{(0)} = 0.05$


In [0]:
from tan.eqnonlin.newton import newton


df=lambda x: 1000*((1+x)**5*(1-5*x) - 1)/(x**2)
zero,res,niter,inc=newton(f,df,.05,1e-5,1000)
print(f"Zero: {zero}, Residual: {res}, Iterations: {niter}")


on trouve le même résultat après seulement 3 itérations.
On peut donc conclure que le taux cherché est $6.14\%$.
*Tiges encastrées*\
On veut tracer le graphe de l’angle $\theta$ en fonction de $\omega$ pour $0\leq\omega\leq\pi$.
On considère les valeurs suivantes pour $a_i$: $a_1 = 10\,cm$, $a_2 = 13\,cm$, $a_3 = 8\,cm$, $a_4 = 10\,cm$.
Pour tout $\omega$ on doit résoudre l’équation non-linéaire


$$
    f(\theta) = \frac{a_1}{a_2}cos(\omega) - \frac{a_1}{a_4}cos(\theta) -
    cos(\omega-\theta) + \frac{a_1^2 + a_2^2 - a_3^2 + a_4^2}{2a_2a_4} =     0
$$


In [0]:
# Constants
a1, a2, a3, a4, omega = 10, 13, 8, 10, np.pi/3

# Define the function
f = lambda x: (a1/a2)*np.cos(omega) - (a1/a4)*np.cos(x) - np.cos(omega-x) + (a1**2 + a2**2 - a3**2 + a4**2) / (2*a2*a4)

# Define the x range
x = np.arange(-np.pi, np.pi, 0.01)

# Calculate y values
y = f(x)

# Create the plot
fig = go.Figure(data=go.Scatter(x=x, y=y, mode='lines'))

# Add grid and customize layout
fig.update_layout(
    title='Plot of f(theta) for omega = pi/3',
    xaxis_title='theta',
    yaxis_title='f(theta)',
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True)
)

# Show the plot
fig.show()


on peut tracer le graphe de la la fonction $f(\theta)$ en correspondance de la valeur $\omega=\pi/3$.

On voit bien qu’il y a deux zéros (c-à-d deux configurations possibles).

On choisit 101 valeurs pour $\omega$: $\omega_k = k \frac{\pi}{100}, \; k=0,\ldots,100$ et on résout l’équation [[sol_ex2]](#sol_ex2) pour chaque valeur $\omega_k$ avec la méthode de Newton (avec tolérance de $10^{-5}$) en partant de deux valeurs initiales différentes $x^{(0)} = -0.1$ et $x^{(0)} = 2\pi/3$.
La figure suivante montre les deux solutions obtenues en correspondance de chaque angle $\omega<\omega^*$.
Pour $\omega>2.6358$ l’algorithme ne converge plus car il n’y a plus de configurations possibles.


![image](eqnonlin/Newex2_2nonlin.ps)


Voici le code python qu’on a utilisé pour obtenir les résultats montrés dans la figure prédente:


In [0]:
from tan.eqnonlin.newton import newton
import numpy as np
import plotly.graph_objs as go

# Constants
a1, a2, a3, a4 = 10, 13, 8, 10
n = 100
x01, x02 = -0.1, 2 * np.pi / 3
nmax = 100

# Define the function and its derivative
f = lambda x, w, a1, a2, a3, a4: (a1/a2)*np.cos(w) - (a1/a4)*np.cos(x) - np.cos(w-x) + ((a1**2 + a2**2 - a3**2 + a4**2) / (2*a2*a4))
df = lambda x, w, a1, a2, a3, a4: (a1/a4)*np.sin(x) - np.sin(w-x)

# Prepare data storage
omega = np.linspace(0, 3*np.pi/2, n+1)
theta1 = np.zeros(n+1)
theta2 = np.zeros(n+1)

# Perform Newton's method for each w
for i, w in enumerate(omega):
    theta1[i], _, niter,    _ = newton(f, df, x01, 1e-5, nmax, w, a1, a2, a3, a4)  # Ensure that your newton function can handle the extra parameter w
    print(f"omega: {w}, theta1: {theta1[i]}") if niter < 100 else None
    theta2[i], _, niter, _ = newton(f, df, x02, 1e-5, nmax, w, a1, a2, a3, a4)
    print(f"omega: {w}, theta2: {theta2[i]}") if niter < 100 else None


Nous pouvons maintenant tracer les deux solutions en fonction de $\omega$:


In [0]:
# Plot the results using Plotly
fig = go.Figure()

# Add traces for theta1 and theta2
fig.add_trace(go.Scatter(x=omega, y=theta1, mode='lines', name='theta1', line=dict(color='blue', dash='dot')))
fig.add_trace(go.Scatter(x=omega, y=theta2, mode='lines', name='theta2', line=dict(color='green', dash='solid')))

# Add grid and customize layout
fig.update_layout(
    title='Plot of Theta1 and Theta2 vs. Omega',
    xaxis_title='Omega',
    yaxis_title='Theta',
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True)
)

# Show the plot
fig.show()


*Concentration de CO$_2$*\
On considère le dioxyde de carbone (CO$_2$), pour lequel $a=0.401 \; Pa \, m^6$ et $b=42.7\cdot 10^{-6} \; m^3$.

On cherche le volume occupé par $N=1000$ molécules de CO$_2$ à température $T=300 \; K$ et pression $p=3.5 \cdot 10^7 \; Pa$, en sachant que la constante de Boltzmann est latexmath:[k=1.3806503 \cdot
10^{-23} \; Joule \, K^{-1}].

On trace le graphe de la fonction


$$
f(V) = \left[ p + a \left( \frac{N}{V} \right)^2 \right] (V-Nb) - kNT
$$

pour $V>0 \,$.
On ne considère pas les valeurs de $V<0$, qui n’ont pas de sens physique, puisque $V$ représente le volume du gaz.


In [0]:
# Constants
a, b, p, T, N, k = 0.401, 42.7e-6, 3.5e7, 300, 1000, 1.3806503e-23

# Define the function
f = lambda x,p,T,a,b,N,k: (p + a * (N / x)**2) * (x - N*b) - k * N * T

# Define the x range
x = np.arange(0.03, 0.1, 0.001)

# Calculate y values
y = f(x,p,T,a,b,N,k)

# Create the plot
fig = go.Figure(data=go.Scatter(x=x, y=y, mode='lines'))

# Add grid and customize layout
fig.update_layout(
    title='Plot of f(V)',
    xaxis_title='V',
    yaxis_title='f(V)',
    xaxis=dict(showgrid=True),
    yaxis=dict(showgrid=True)
)

# Show the plot
fig.show()


on obtient le graphe de la fonction $f(V)$:

On voit qu’il y a un zéro pour $0.03<V<0.1$.
Si on applique la méthode de dichotomie sur l’intervalle $[0.03,0.1]$ avec une tolérance de $10^{-12}$:


In [0]:
zero,res,niter,inc=bisection(f,0.03,0.1,1e-12,1000,p,T,a,b,N,k)
print(f"Zero: {zero}, Residual: {res}, Iterations: {niter}")


On trouve, après $36$ itérations, la valeur $V=0.0427$.

Si, au contraire, on utilise la méthode de Newton avec la même tolérance, à partir du point initial $x^{(0)} = 0.03$,


In [0]:
df=lambda x,p,T,a,b,N,k: -2*a*N**2/(x**3)*(x-N*b)+(p+a*((N/x)**2))
zero,res,niter,inc=newton(f,df,0.03,1e-12,1000,p,T,a,b,N,k)
print(f"Zero: {zero}, Residual: {res}, Iterations: {niter}")


on trouve la même solution après $6$ itérations.

On conclut que le volume $V$ occupé par le gaz est $0.0427 \;m^3$.
